### Baseline models to consider:
 - BART model from the work of SciTech News
 - Zero and Few-shot prompting of LLAMA-3
 - Fine-tuned LLAMA-3
 - BART model extended with conversation between (non) fine-tuned LLAMA-3s

In [1]:
%load_ext autoreload

In [2]:
import os
import sys
import numpy as np

In [3]:
os.environ['TRANSFORMERS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache'
os.environ['HF_DATASETS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache'
os.environ['OPENAI_API_KEY'] = 'zTbZNk16Ik1pZnqLn38ZT3BlbkFJImq3pd7widkr7RzsC771'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append('./src-py')

In [4]:
import tiktoken
import datasets
import json
import os
import pandas as pd
import torch
from tabulate import tabulate

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, pipeline
from peft import LoraConfig, get_peft_model

/mnt/swordfish-pool2/milad/conda-envs/datadreamer/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [5]:
%autoreload
import utils
import prompts
import datadreamer_generation
from llm_based_evaluation import *

In [6]:
ds_path = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/eval_experiment_500/baseline_llama3_gen_conv/'

In [7]:
sample_test = datasets.load_from_disk(ds_path)

In [8]:
sample_test

Dataset({
    features: ['paper_id', 'paper_title', 'paper_text', 'prompt', 'completion', 'pr-article', 'topic', '__index_level_0__', 'generated_conversation', 'conversation'],
    num_rows: 500
})

### Zero and few shot journalistic summarization:

In [9]:
from datadreamer.llms import HFTransformers, ParallelLLM, OpenAI
from transformers import BitsAndBytesConfig

q=BitsAndBytesConfig(load_in_8bit=True)
llama3 = HFTransformers("meta-llama/Meta-Llama-3-8B-Instruct", device='auto', quantization_config=q, dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
tokenizer.pad_token_id = tokenizer.eos_token_id

In [10]:
sample_test = datasets.load_from_disk(ds_path)

In [11]:
ds_path = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/baseline-generation/'

In [12]:
prompt = prompts.baseline_pr_generation
prompt['inputs']['Scientific paper'] = 'paper_text'
baseline_pr_generation_res = datadreamer_generation.generate_pr_articles(llama3, sample_test, ds_path + '/generated-prs/'  + prompt['strategy_name'], 
                                                                         tokenizer, prompt, max_input_tokens=1200, hub_name=None)
baseline_pr_generation_res.save_to_disk(ds_path + '/generated-prs/'  + prompt['strategy_name'] + '_final_ds')

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/baseline-generation//generated-prs/baseline_pr_generationbaseline_pr_generation
[ 🤖 DataDreamer 💤 ] Step 'original_ds' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' will run lazily. 🥱
/local/nlp/milad/code/llms-as-science-communicators/src-py/datadreamer_generation.py:332: UserWarning: You did not specify `total_num_rows`, so we cannot automatically update the progress % for this step. Either specify map(..., total_num_rows=#) or, to disable this warning, specify map(.., auto_progress = False)
  datasource = datasource.map(lambda row: {'inputs_truncated': truncate_text(encoding, row['inputs'], max_input_tokens)})
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map) (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'o


        Please write a press release article to communicate the science presented in the following scientific paper. Your output should be:
        "Press Release Article": "The press release article about the paper"
    


[ 🤖 DataDreamer 💤 ] Step 'pr-writings' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map) (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] [ 🧠 HFTransformers (meta-llama/Meta-Llama-3-8B-Instruct)] Loading...
[ 🤖 DataDreamer 💤 ] [ 🧠 HFTransformers (meta-llama/Meta-Llama-3-8B-Instruct)] Finished loading.
/mnt/swordfish-pool2/milad/conda-envs/datadreamer/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 10 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 20 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 30 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 40 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 5

Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output
Failed to find PR Article in the output


[ 🤖 DataDreamer 💤 ] Step 'zipped(original_ds (map) (map), pr-writings (select_columns))' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'zipped(original_ds (map) (map), pr-writings (select_columns)) (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/baseline-generation//generated-prs/baseline_pr_generationbaseline_pr_generation


Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [13]:
prompt = prompts.baseline_pr_generation_cot
prompt['inputs']['Scientific paper'] = 'paper_text'
baseline_pr_generation_res = datadreamer_generation.generate_pr_articles(llama3, sample_test, ds_path + '/generated-prs/'  + prompt['strategy_name'], 
                                                                         tokenizer, prompt, max_input_tokens=1200, hub_name=None)
baseline_pr_generation_res.save_to_disk(ds_path + '/generated-prs/'  + prompt['strategy_name'] + '_final_ds')

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/baseline-generation//generated-prs/baseline_pr_generation_cotbaseline_pr_generation_cot
[ 🤖 DataDreamer 💤 ] Step 'original_ds' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' will run lazily. 🥱
/local/nlp/milad/code/llms-as-science-communicators/src-py/datadreamer_generation.py:332: UserWarning: You did not specify `total_num_rows`, so we cannot automatically update the progress % for this step. Either specify map(..., total_num_rows=#) or, to disable this warning, specify map(.., auto_progress = False)
  datasource = datasource.map(lambda row: {'inputs_truncated': truncate_text(encoding, row['inputs'], max_input_tokens)})
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map) (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ]


        Please write a press release article to communicate the science presented in the following paper.
        Before generating the press release, think step by step about the social impact of the research paper, the innovative aspects of the paper and how it is different from other research on the same topic, and how to communicate the problem, the approach and the results of the paper in a simple and accessible lanuage. Finally output the press release in the following format:
        "Press Release Article": "The press release article about the paper"
    


[ 🤖 DataDreamer 💤 ] Step 'pr-writings' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map) (map)' finished running lazily. 🎉
/mnt/swordfish-pool2/milad/conda-envs/datadreamer/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 10 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 20 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 30 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 40 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 50 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 60 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 70 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 80 

Failed to find PR Article in the output
Failed to find PR Article in the output


[ 🤖 DataDreamer 💤 ] Step 'zipped(original_ds (map) (map), pr-writings (select_columns))' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'zipped(original_ds (map) (map), pr-writings (select_columns)) (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/baseline-generation//generated-prs/baseline_pr_generation_cotbaseline_pr_generation_cot


Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [14]:
prompt = prompts.baseline_pr_generation_w_conv
prompt['inputs']['Scientific paper'] = 'paper_text'
baseline_pr_generation_res = datadreamer_generation.generate_pr_articles(llama3, sample_test, ds_path + '/generated-prs/'  + prompt['strategy_name'], 
                                                                         tokenizer, prompt, max_input_tokens=1200, hub_name=None)
baseline_pr_generation_res.save_to_disk(ds_path + '/generated-prs/'  + prompt['strategy_name'] + '_final_ds')

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/baseline-generation//generated-prs/baseline_pr_generation_w_convbaseline_pr_generation_w_conv
[ 🤖 DataDreamer 💤 ] Step 'original_ds' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' will run lazily. 🥱
/local/nlp/milad/code/llms-as-science-communicators/src-py/datadreamer_generation.py:332: UserWarning: You did not specify `total_num_rows`, so we cannot automatically update the progress % for this step. Either specify map(..., total_num_rows=#) or, to disable this warning, specify map(.., auto_progress = False)
  datasource = datasource.map(lambda row: {'inputs_truncated': truncate_text(encoding, row['inputs'], max_input_tokens)})
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map) (map)' is running. ⏳
[ 🤖 DataDream


        Please write a press release article to communicate the science presented in the following paper.
        Before generating the press release, generate a conversation between the paper's author and a journalist where they discuss the social impact of the research paper, the innovative aspects of the paper and how it is different from other research on the same topic. The author explains the paper in a very simple and accessible language to the journalist. Finally output the press release in the following format:
        "Press Release Article": "The press release article about the paper"        
    


[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map) (map)' finished running lazily. 🎉
/mnt/swordfish-pool2/milad/conda-envs/datadreamer/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 10 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 20 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 30 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 40 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 50 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 60 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 70 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 80 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progr

Failed to find PR Article in the output


[ 🤖 DataDreamer 💤 ] Step 'zipped(original_ds (map) (map), pr-writings (select_columns))' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'zipped(original_ds (map) (map), pr-writings (select_columns)) (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/baseline-generation//generated-prs/baseline_pr_generation_w_convbaseline_pr_generation_w_conv


Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [48]:
sample_test = sample_test.map(lambda row: {'generated-conversation-as-str': '\n'.join(['{}: {}'.format('Journalist', x['content']) if x['role'] == 'assistant' else '{}: {}'.format('Researcher', x['content']) for x in row['generated_conversation'][1:]])})

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [50]:
prompt = prompts.pr_generation_by_conv_summarization
prompt['inputs']['Conversation'] = 'generated-conversation-as-str'
pr_generation_w_conv_res = baselines.generate_pr_articles(llama3, sample_test, ds_path, tokenizer, prompt, hub_name=None)
pr_generation_w_conv_res.save_to_disk(ds_path + prompt['strategy_name'] + '_final_ds')

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/baseline-generation/pr_generation_by_conv_summarization
[ 🤖 DataDreamer 💤 ] Step 'original_ds' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' will run lazily. 🥱
/home/ma4608/communicating-science-to-the-public/src/baselines.py:77: UserWarning: You did not specify `total_num_rows`, so we cannot automatically update the progress % for this step. Either specify map(..., total_num_rows=#) or, to disable this warning, specify map(.., auto_progress = False)
  datasource = datasource.map(lambda row: {'inputs_len': len(encoding.encode(row['inputs']))})
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map) (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map) (map)' will run lazily. 🥱
/home/ma4608/communicating-s


        Please write a press release article to communicate the science presented in the following paper. The press-release should summarize the main points in the given conversation. The output should have the following format:
        "Press Release Article": "The press release article about the paper"        
    


[ 🤖 DataDreamer 💤 ] Step 'original_ds (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map) (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'original_ds (map) (map) (filter)' finished running lazily. 🎉
/mnt/swordfish-pool2/milad/conda-envs/datadreamer/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 10 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 20 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 30 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 40 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 50 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 60 row(s) 🔄
[ 🤖 DataDreamer 💤 ] Step 'pr-writings' progress: 70 row(s) 🔄
[ 🤖 DataDreamer 💤 

Failed to find PR Article in the output


[ 🤖 DataDreamer 💤 ] Step 'zipped(original_ds (map) (map) (filter), pr-writings (select_columns))' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'zipped(original_ds (map) (map) (filter), pr-writings (select_columns)) (map)' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/baseline-generation/pr_generation_by_conv_summarization


Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [52]:
#clm='parsed-pr-article'
clm='gen-pr'
row=0
print(baseline_pr_generation_res[clm][row])
print('++++++++++++++')
print(baseline_pr_generation_cot_res[clm][row])
print('++++++++++++++')
print(baseline_pr_generation_w_conv_res[clm][row])
print('++++++++++++++')
print(pr_generation_w_conv_res[clm][row])

**Press Release Article**

**Scientists Search for Dark Matter with Global Network of Optical Magnetometers**

A team of scientists has conducted a groundbreaking search for dark matter using a global network of optical magnetometers, known as GNOME (Global Network of Optical Magnetometers for Exotic physics searches). The search, published in a recent scientific paper, aimed to detect the presence of axion-like particles, a type of dark matter candidate, by monitoring the Earth's passage through hypothetical domain walls.

Dark matter is a mysterious substance that makes up approximately 80% of the universe's mass, yet its nature remains unknown. Axion-like particles are a popular theory for explaining dark matter's existence. These particles can form stable, macroscopic field configurations, such as topological defects, which could concentrate dark matter density into distinct, compact regions.

The GNOME network, comprising over a dozen optical atomic magnetometers located worldwide

In [26]:
# Use the two trained llms
# ds_path = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/llama3-trained-on-llama3-for-1-epochs/'

# q=BitsAndBytesConfig(load_in_8bit=True)
# llama3 = HFTransformers("meta-llama/Meta-Llama-3-8B-Instruct", device='auto', adapter_name='miladalsh/llama3-trained-on-llama3-for-1-epochs', 
#                         quantization_config=q, dtype=torch.bfloat16)
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
# tokenizer.pad_token_id = tokenizer.eos_token_id

# prompt = prompts.baseline_pr_generation_w_conv
# prompt['inputs']['Scientific paper'] = 'sc-intro'
# pr_generation_w_conv_res = baselines.generate_pr_articles(llama3, sample_test, ds_path, tokenizer, prompt, hub_name=None)
# pr_generation_w_conv_res.save_to_disk(ds_path + prompt['strategy_name'] + 'final_ds')

### Evaluation

In [10]:
ds_path = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases/'

In [11]:
baseline_pr_generation_res = datasets.load_from_disk(ds_path + '/baseline-generation/' + 'baseline_pr_generation_final_ds')
baseline_pr_generation_cot_res = datasets.load_from_disk(ds_path + '/baseline-generation/' + 'baseline_pr_generation_cot_final_ds')
baseline_pr_generation_conv_res = datasets.load_from_disk(ds_path + '/baseline-generation/' + 'baseline_pr_generation_w_conv_final_ds')
pr_generation_conv_res = datasets.load_from_disk(ds_path + '/baseline-generation/' + 'pr_generation_by_conv_summarization_final_ds')

#### Basic Eval:

In [12]:
eval_results = {}

eval_results['baseline_pr_generation'] = utils.evaluate_text_similarity(baseline_pr_generation_res['parsed-pr-article'],  baseline_pr_generation_res['pr-article'])
eval_results['baseline_pr_generation_cot'] = utils.evaluate_text_similarity(baseline_pr_generation_cot_res['parsed-pr-article'],  baseline_pr_generation_cot_res['pr-article'])
eval_results['baseline_pr_generation_w_conv'] = utils.evaluate_text_similarity(baseline_pr_generation_conv_res['parsed-pr-article'],  baseline_pr_generation_conv_res['pr-article'])
eval_results['pr_generation_w_conv'] = utils.evaluate_text_similarity(pr_generation_conv_res['parsed-pr-article'],  pr_generation_conv_res['pr-article'])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
#baseline_pr_generation_cot_res['gen-pr'][0]

In [24]:
#baseline_pr_generation_cot_res['parsed-pr-article'][0]

In [13]:
print(tabulate(
    [[name] + list(eval_res.values())[:3] for name, eval_res in eval_results.items()],
    headers=['Model', 'Rouge-1', 'Rouge-L', 'BERT-f1']
))

Model                            Rouge-1    Rouge-L    BERT-f1
-----------------------------  ---------  ---------  ---------
baseline_pr_generation             0.434      0.181      0.837
baseline_pr_generation_cot         0.435      0.178      0.835
baseline_pr_generation_w_conv      0.38       0.167      0.834
pr_generation_w_conv               0.405      0.172      0.829


#### LLM-based evalaution:

In [13]:
grdound_truth_ds = baseline_pr_generation_res.map(lambda row: {'parsed-pr-article': row['pr-summary-and-article']})

In [66]:
press_release_datasets = {
    'baseline_pr_generation' : [ datasets.load_from_disk(ds_path + '/baseline-generation/' + 'baseline_pr_generation_final_ds'), ds_path + '/baseline-generation/' + 'baseline_pr_generation_final_ds'],
    'baseline_pr_generation_cot': [datasets.load_from_disk(ds_path + '/baseline-generation/' + 'baseline_pr_generation_cot_final_ds'),ds_path + '/baseline-generation/' + 'baseline_pr_generation_cot_final_ds'],
    'baseline_pr_generation_w_conv': [datasets.load_from_disk(ds_path + '/baseline-generation/' + 'baseline_pr_generation_w_conv_final_ds'), ds_path + '/baseline-generation/' + 'baseline_pr_generation_w_conv_final_ds'],
    'pr_generation_w_conv': [datasets.load_from_disk(ds_path + '/baseline-generation/' + 'pr_generation_by_conv_summarization_final_ds'), ds_path + '/baseline-generation/' + 'pr_generation_by_conv_summarization_final_ds'],
    'gt_press_release' : [grdound_truth_ds, ds_path + '/gt-press-release'],
}

In [ ]:
prompts_to_eval = [prompts.pr_clarity_eval_prompt, prompts.pr_scientific_context_eval_prompt, prompts.pr_societal_context_eval_prompt]

llm_eval_results = llm_based_evaluation(prompts_to_eval, press_release_datasets)

Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases//baseline-generation/baseline_pr_generation_final_ds from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases//baseline-generation/baseline_pr_generation_cot_final_ds from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases//baseline-generation/baseline_pr_generation_w_conv_final_ds from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/generated-press-releases//baseline-generation/pr_generation_by_conv_summarization_final_ds from already saved file


  0%|                                                                                        | 0/494 [00:00<?, ?it/s]

In [68]:
print(tabulate(
        [[name] + get_llm_avg_scores(res) for name, res in llm_eval_results.items()],
        headers=['Prompt', 'Scientific Context', 'Social Context', 'Accessibility', 'Relevance', 'Avg']
    ))

Prompt                           Scientific Context    Social Context    Accessibility    Relevance
-----------------------------  --------------------  ----------------  ---------------  -----------
baseline_pr_generation                         1.74              1.95             3.6          2.43
baseline_pr_generation_cot                     1.75              1.99             3.8          2.51
baseline_pr_generation_w_conv                  1.7               2.04             3.86         2.53
pr_generation_w_conv                           1.64              2.06             3.38         2.36
gt_press_release                               2.48              2.22             4.39         3.03
